In [ ]:
import requests
import time 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

client_id = 186247
client_secret = "0ae223d35225bb2f40f9b44a9a70ae18fd29169b"
code = "520a81df6848cfb6ab36093fe25a46438caf354a"

url = "https://www.strava.com/oauth/token"

payload = {
    "client_id": client_id,
    "client_secret": client_secret,
    "code": code,
    "grant_type": "authorization_code"
}

tokens = requests.post(url, data=payload).json()
print(tokens)

refresh_token = tokens["refresh_token"]

payload = {
    "client_id": client_id,
    "client_secret": client_secret,
    "grant_type": "refresh_token",
    "refresh_token": refresh_token
}

refreshed = requests.post(url, data=payload).json()

access_token = tokens["access_token"]

headers = {"Authorization": f"Bearer {access_token}"}

url = "https://www.strava.com/api/v3/athlete/activities"

activities_list = []

page = 1
for _ in range(12):
    r = requests.get(url, headers=headers, params={"page":page, "per_page": 100})
    activities = r.json()
    activities_list.extend(activities)
    page+=1

In [61]:
pd.value_counts([len(dic.keys()) for dic in activities_list])

/var/folders/87/rbsvb18s08x2_20r3q58gdjw0000gn/T/ipykernel_97782/2099348061.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts([len(dic.keys()) for dic in activities_list])
/var/folders/87/rbsvb18s08x2_20r3q58gdjw0000gn/T/ipykernel_97782/2099348061.py:1: FutureWarning: value_counts with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  pd.value_counts([len(dic.keys()) for dic in activities_list])


48    140
55    123
50    120
44     12
49      1
Name: count, dtype: int64

In [76]:

all_keys = set().union(*[dic.keys() for dic in activities_list])

df = pd.DataFrame([{key: dic.get(key, None) for key in all_keys} for dic in activities_list])
df['date_str'] = df['start_date'].apply(lambda x: pd.Timestamp(x).date())
df['time_mins'] = df['elapsed_time'].apply(lambda x: x / 60)

0      174.4
1      166.0
2      112.9
3      158.7
4      172.3
       ...  
391      NaN
392      NaN
393      NaN
394      NaN
395      NaN
Name: average_heartrate, Length: 396, dtype: float64